In [1]:
import pandas as pd
import math
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from collections import defaultdict
from scipy.stats.stats import pearsonr
import seaborn as sn

In [2]:
df = pd.read_csv('dataset/matches_datacleaning.csv', index_col = 0)
pd.set_option('display.max_columns', None) # to visualize all the columns
df.info()


/Users/domenico/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


<class 'pandas.core.frame.DataFrame'>
Int64Index: 184890 entries, 0 to 186072
Data columns (total 51 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   tourney_id          184890 non-null  object 
 1   tourney_name        184890 non-null  object 
 2   surface             184728 non-null  object 
 3   draw_size           184890 non-null  float64
 4   tourney_level       184890 non-null  object 
 5   tourney_date        184890 non-null  object 
 6   match_num           184890 non-null  float64
 7   winner_id           184890 non-null  float64
 8   winner_entry        25701 non-null   object 
 9   winner_name         184890 non-null  object 
 10  winner_hand         184890 non-null  object 
 11  winner_ht           48560 non-null   float64
 12  winner_ioc          184890 non-null  object 
 13  winner_age          182080 non-null  float64
 14  loser_id            184890 non-null  float64
 15  loser_entry         43927 non-null

## new database building

In [3]:
name = pd.concat([df['winner_name'], df['loser_name']]) 
#name.rename(columns={0: 'name'}, inplace = True)
name.drop_duplicates(inplace=True)


new_df = pd.DataFrame(name)
new_df.rename(columns={0: 'name'}, inplace = True)
print(new_df)

                      name
0            KEI NISHIKORI
1          DANIIL MEDVEDEV
3       JO-WILFRIED TSONGA
5            JEREMY CHARDY
8           ALEX DE MINAUR
...                    ...
185021    DILARA SULTANOVA
185560    MADELEINE JESSUP
185780          KATIE CODD
185781           AMY HUANG
185940     CHLOE HENDERSON

[10098 rows x 1 columns]


In [4]:
def create_new_colums(winner_attr, loser_attr, attr):
    col1=df[['winner_name',winner_attr]]
    col2=df[['loser_name',loser_attr]]
    col2.rename(columns={'loser_name': 'winner_name'}, inplace = True)
    col2.rename(columns={loser_attr: winner_attr}, inplace = True)
    name2 = col1.append(col2) 
    name2.rename(columns={'winner_name': 'name',winner_attr:attr}, inplace = True)
    name2.drop_duplicates(inplace=True)
    return name2


## Adding hand of players

In [5]:
value=create_new_colums('winner_hand','loser_hand','hand')
value.at[180480, 'hand'] = 'R'
value.drop_duplicates(inplace=True)


/Users/domenico/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [6]:
new_df['hand']=value.hand
print(new_df)

                      name hand
0            KEI NISHIKORI    R
1          DANIIL MEDVEDEV    R
3       JO-WILFRIED TSONGA    R
5            JEREMY CHARDY    R
8           ALEX DE MINAUR    R
...                    ...  ...
185021    DILARA SULTANOVA    U
185560    MADELEINE JESSUP    U
185780          KATIE CODD    U
185781           AMY HUANG    U
185940     CHLOE HENDERSON    U

[10098 rows x 2 columns]


In [7]:
#new_df.reset_index(inplace=True)
#del new_df['index']
#print(new_df)

## adding sex of players

In [8]:
value=create_new_colums('winner_gender','loser_gender','gender')


/Users/domenico/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [9]:
print(value.gender)

0         M
1         M
3         M
5         M
8         M
         ..
185021    F
185560    F
185780    F
185781    F
185940    F
Name: gender, Length: 10098, dtype: object


In [10]:
new_df['gender']=value.gender
print(new_df)

                      name hand gender
0            KEI NISHIKORI    R      M
1          DANIIL MEDVEDEV    R      M
3       JO-WILFRIED TSONGA    R      M
5            JEREMY CHARDY    R      M
8           ALEX DE MINAUR    R      M
...                    ...  ...    ...
185021    DILARA SULTANOVA    U      F
185560    MADELEINE JESSUP    U      F
185780          KATIE CODD    U      F
185781           AMY HUANG    U      F
185940     CHLOE HENDERSON    U      F

[10098 rows x 3 columns]


## ioc of players

In [11]:
value=create_new_colums('winner_ioc','loser_ioc','ioc')
value.drop_duplicates(inplace=True)

new_df['ioc']=value.ioc
print(new_df)


/Users/domenico/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


                      name hand gender  ioc
0            KEI NISHIKORI    R      M  JPN
1          DANIIL MEDVEDEV    R      M  RUS
3       JO-WILFRIED TSONGA    R      M  FRA
5            JEREMY CHARDY    R      M  FRA
8           ALEX DE MINAUR    R      M  AUS
...                    ...  ...    ...  ...
185021    DILARA SULTANOVA    U      F  TKM
185560    MADELEINE JESSUP    U      F  USA
185780          KATIE CODD    U      F  USA
185781           AMY HUANG    U      F  USA
185940     CHLOE HENDERSON    U      F  USA

[10098 rows x 4 columns]


In [12]:

LoserId_name = df.groupby(['winner_name']).winner_age.nunique() # it contains each value of TourneyId and the number of Tourneyname values it is associated with 
LoserId_name = LoserId_name[LoserId_name > 1] # it contains the TourneyId values associated with more than one Tourneyname value

df[df.winner_name.isin(LoserId_name.index)].groupby(['winner_name']).winner_age.unique()



winner_name
ABBIE MYERS                        [23.0, 22.0, 24.0, 25.0, 26.0]
ABHINAV SANJEEV SHANMUGAM                      [18.0, 19.0, 21.0]
ABIGAIL DESIATNIKOV                                  [17.0, 15.0]
ABIGAIL RENCHELI                                     [17.0, 18.0]
ABIGAIL TERE APISAH                [25.0, 24.0, 26.0, 27.0, 28.0]
                                             ...                 
ZOE KRUGER                   [15.0, 14.0, 16.0, 17.0, 19.0, 18.0]
ZOE RICHARD                              [19.0, 20.0, 21.0, 22.0]
ZOZIYA KARDAVA                     [16.0, 17.0, 18.0, 20.0, 19.0]
ZSOMBOR PIROS                            [18.0, 19.0, 22.0, 20.0]
ZUZANA ZLOCHOVA              [27.0, 28.0, 26.0, 30.0, 29.0, 31.0]
Name: winner_age, Length: 3643, dtype: object

In [19]:
gg=df

print(gg['tourney_date'])
#gg['tourney_date'] = pd.to_datetime(gg['tourney_date'], format='%Y-%m-%d')
#print((gg['tourney_date'].str[:4].astype('int32')-gg['winner_age']))
#var = df.groupby(['winner_name'])['winner_age','tourney_date']
gg['winner_birth']=(gg['tourney_date'].str[:4].astype('int32')-gg['winner_age'])
gg['loser_birth']=(gg['tourney_date'].str[:4].astype('int32')-gg['loser_age'])
#print(var)
print()
#in totale abbiamo 6 valori da salvare, il problema è come farlo in automatico? scegliamo di farlo manualmente per ogni caso
#var = var.apply(lambda x: x[]  if x['winner_age'].size>1 else np.nan, axis=1)



0         2018-12-31
1         2018-12-31
2         2018-12-31
3         2018-12-31
4         2018-12-31
             ...    
186068    2020-11-09
186069    2020-11-09
186070    2020-11-09
186071    2020-11-09
186072    2020-11-09
Name: tourney_date, Length: 184890, dtype: object



In [55]:
#print(gg.groupby(['winner_name']).birth.min())
winner_birth=gg.groupby(['winner_name']).birth.min()
#print(gg.groupby(['loser_name']).birth.min())
loser_birth=gg.groupby(['loser_name']).birth.min()

winner_birth = pd.DataFrame(winner_birth)
winner_birth.reset_index(inplace=True)

loser_birth = pd.DataFrame(loser_birth)
loser_birth.reset_index(inplace=True)

winner_birth.rename(columns={'winner_name': 'name'}, inplace = True)
loser_birth.rename(columns={'loser_name': 'name'}, inplace = True)

name=winner_birth.append(loser_birth)

#name.dropna(inplace=True)
name.drop_duplicates(inplace=True)

name=name.groupby(name['name']).birth.min()

tab = pd.DataFrame(name)
tab.reset_index(inplace=True)

print(tab)


                      name   birth
0                AADA INNA  1999.0
1           AALISHA ALEXIS  1995.0
2           AALIYA EBRAHIM  1992.0
3          AALIYAH HOHMANN  1996.0
4           AALYKA EBRAHIM  1998.0
...                    ...     ...
10093      ZUZANA ZALABSKA  1985.0
10094      ZUZANA ZLOCHOVA  1983.0
10095      ZUZANNA BEDNARZ  1997.0
10096  ZUZANNA SZCZEPANSKA  1992.0
10097   ZVONIMIR ORESKOVIC  1993.0

[10098 rows x 2 columns]


In [54]:
new_df=new_df.sort_values(by=['name'])
new_df.reset_index(inplace=True)
del new_df['index']
new_df['birth']=tab.birth

print(new_df)


                      name hand gender  ioc   birth
0                AADA INNA    U      F  FIN  1999.0
1           AALISHA ALEXIS    U      F  TTO  1995.0
2           AALIYA EBRAHIM    R      F  IND  1992.0
3          AALIYAH HOHMANN    U      F  DEU  1996.0
4           AALYKA EBRAHIM    U      F  IND  1998.0
...                    ...  ...    ...  ...     ...
10093      ZUZANA ZALABSKA    R      F  CZE  1985.0
10094      ZUZANA ZLOCHOVA    R      F  SVK  1983.0
10095      ZUZANNA BEDNARZ    U      F  POL  1997.0
10096  ZUZANNA SZCZEPANSKA    R      F  POL  1992.0
10097   ZVONIMIR ORESKOVIC    R      M  CRO  1993.0

[10098 rows x 5 columns]


LoserId_name = gg.groupby(['name']).birth.nunique() # it contains each value of TourneyId and the number of Tourneyname values it is associated with 
LoserId_name = LoserId_name[LoserId_name > 1] # it contains the TourneyId values associated with more than one Tourneyname value

gg[gg.winner_name.isin(LoserId_name.index)].groupby(['name']).birth.unique()

